### Populate PEMS metadata in SQL RDS database

Daniel Roten <daroten@eng.ucsd.edu>

In [3]:
!ls ../data/traffic/meta/2021/d10

d10_text_meta_2020_12_02.txt


In [4]:
import pandas as pd
import numpy as np

In [5]:
from glob import glob

In [6]:
flist=glob("../data/traffic/meta/2021/*/*.txt")

In [7]:
flist[0]

'../data/traffic/meta/2021/d10/d10_text_meta_2020_12_02.txt'

Testing on one file.  Separator is tab.  Only the first 18 rows are useable.

In [8]:
afile="../data/traffic/meta/2021/d11/d11_text_meta_2020_10_02.txt"
#!head -945 $afile | tail -3

df=pd.read_csv(afile, sep="\t", usecols=range(18), index_col=0)#, warn_bad_lines=True, error_bad_lines=False)
df.head(n=1)

,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude,Length,Type,Lanes,Name,User_ID_1,User_ID_2,User_ID_3,User_ID_4
ID,,,,,,,,,,,,,,,,,
1100313,5,N,11,73,66000.0,22.324,22.215,32.79154,-117.20716,NaN,FR,1,CLAIREMONT DR,1,NaN,NaN,NaN


In [9]:
dflist=[pd.read_csv(afile, sep="\t", usecols=range(18)) for afile in flist]

In [10]:
df=pd.concat(dflist, axis=0)

In [11]:
print("Total number of rows:", len(df))
print("Number of unique sensor IDs:", len(df.drop_duplicates(subset="ID")))
print("Number of unique rows:", len(df.drop_duplicates()))

Total number of rows: 92414
Number of unique sensor IDs: 18504
Number of unique rows: 19034


There's a whole bunch of duplicate entries.  Not sure what they mean and which one to use if there are several.

In [12]:
df.groupby("ID").agg({"Fwy": "count"}).sort_values("ID", ascending=False).head()

,Fwy
ID,
506800143,3
506800142,3
506800141,3
506800132,3
506800131,3


In [13]:
mask=df["ID"] == 506800131
df[mask]

,ID,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude,Length,Type,Lanes,Name,User_ID_1,User_ID_2,User_ID_3,User_ID_4
482,506800131,68,E,5,53,NaN,17.257,17.461,36.625285,-121.680718,0.814,ML,2,RESERVATION & RIVER RD HWY 68 EB,1,NaN,NaN,NaN
481,506800131,68,E,5,53,NaN,17.257,17.461,36.625285,-121.680718,0.814,ML,2,RESERVATION & RIVER RD HWY 68 EB,1,NaN,NaN,NaN
481,506800131,68,E,5,53,NaN,17.257,17.461,36.625285,-121.680718,0.814,ML,2,RESERVATION & RIVER RD HWY 68 EB,1,NaN,NaN,NaN


In [14]:
adf=df.drop_duplicates(subset="ID", keep="last")
adf.set_index("ID", drop=True, inplace=True)

In [15]:
adf.head()

,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude,Length,Type,Lanes,Name,User_ID_1,User_ID_2,User_ID_3,User_ID_4
ID,,,,,,,,,,,,,,,,,
1000110,5,S,10,77,NaN,R21.407,466.899,37.881396,-121.278917,0.749,ML,3,N/O Mathews Rd,NaN,NaN,NaN,NaN
1000210,5,S,10,77,NaN,R20.8789,466.371,37.873818,-121.277900,0.478,ML,3,N/O Eldorado St,NaN,NaN,NaN,NaN
1000310,5,S,10,77,NaN,R20.45,465.942,37.867677,-121.278858,0.389,ML,3,N/O Eldorado St,NaN,NaN,NaN,NaN
1000410,5,S,10,77,NaN,R20.1,465.592,37.862729,-121.280287,0.397,ML,3,N/O Roth Rd,NaN,NaN,NaN,NaN
1000510,5,S,10,77,40704.0,R19.655,465.147,37.856421,-121.282010,0.474,ML,3,N/O Roth Rd,NaN,NaN,NaN,NaN


### Note:

I'm not sure if what these fields mean and if they are necessary:
- State_PM
- Abs_PM
- USER_ID_1
- USER_ID_2
- USER_ID_3
- USER_ID_4

Leaving them out for now, but should be easy to add them if needed.

In [16]:
adf["Length"].max()

5.0

In [17]:
#Figuring out allowed values (constraints) for these fields:
print(adf["Dir"].unique())
print(adf["Type"].unique())

['S' 'W' 'E' 'N']
['ML' 'CH' 'OR' 'FR' 'HV' 'FF' 'CD']


In [18]:
adf["Lanes"].unique()

array([3, 5, 2, 4, 1, 6, 7, 8])

In [19]:
adf.columns

Index(['Fwy', 'Dir', 'District', 'County', 'City', 'State_PM', 'Abs_PM',
       'Latitude', 'Longitude', 'Length', 'Type', 'Lanes', 'Name', 'User_ID_1',
       'User_ID_2', 'User_ID_3', 'User_ID_4'],
      dtype='object')

In [20]:
#Creating limited dataframe with columns written to database
#Making column and index labels consistent with DB table
scols=["Fwy","Dir","District","County","City","Latitude","Longitude","Length","Type","Lanes","Name"]
ddf=adf[scols]

columns=ddf.columns.str.lower().tolist()
columns[1]="direc"
columns[8]="stype"
ddf.columns=columns
ddf.index.rename("sid", inplace=True)

In [21]:
ddf.head()

,fwy,direc,district,county,city,latitude,longitude,length,stype,lanes,name
sid,,,,,,,,,,,
1000110,5,S,10,77,NaN,37.881396,-121.278917,0.749,ML,3,N/O Mathews Rd
1000210,5,S,10,77,NaN,37.873818,-121.277900,0.478,ML,3,N/O Eldorado St
1000310,5,S,10,77,NaN,37.867677,-121.278858,0.389,ML,3,N/O Eldorado St
1000410,5,S,10,77,NaN,37.862729,-121.280287,0.397,ML,3,N/O Roth Rd
1000510,5,S,10,77,40704.0,37.856421,-121.282010,0.474,ML,3,N/O Roth Rd


In [23]:
#ddf["city"].fillna(value=NULL)

In [22]:
import getpass
import sqlalchemy as sal

In [23]:
endpoint = "capstone.clihskgj8i7s.us-west-2.rds.amazonaws.com"
user="group3"
db="db1"
pw=getpass.getpass("Enter database password")

Enter database password········


In [24]:
engine = sal.create_engine('postgresql://%s:%s@%s/%s' % (user, pw, endpoint, db))

In [26]:
#engine.execute("delete from pemslocs;")

In [27]:
pd.read_sql("select * from pemslocs", engine)

,sid,fwy,direc,district,county,city,latitude,longitude,length,stype,lanes,name


In [34]:
ddf.to_sql("pemslocs", engine, if_exists="append")

In [35]:
pd.read_sql("select * from pemslocs limit 10", engine, index_col="sid")

,fwy,direc,district,county,city,latitude,longitude,length,stype,lanes,name
sid,,,,,,,,,,,
1000110,5,S,10,77,NaN,37.881396,-121.278917,0.749,ML,3,N/O Mathews Rd
1000210,5,S,10,77,NaN,37.873818,-121.277900,0.478,ML,3,N/O Eldorado St
1000310,5,S,10,77,NaN,37.867677,-121.278858,0.389,ML,3,N/O Eldorado St
1000410,5,S,10,77,NaN,37.862729,-121.280287,0.397,ML,3,N/O Roth Rd
1000510,5,S,10,77,40704.0,37.856421,-121.282010,0.474,ML,3,N/O Roth Rd
1000610,5,S,10,77,40704.0,37.849282,-121.283757,0.273,ML,3,S/O Roth Rd
1000710,5,S,10,77,40704.0,37.848654,-121.283900,0.249,ML,3,N/O Lathrop Rd
1000810,5,S,10,77,40704.0,37.842193,-121.285444,0.470,ML,3,N/O Lathrop Rd
1000910,5,S,10,77,40704.0,37.835254,-121.287050,0.686,ML,3,N/O Lathrop Rd
